# Parameter Set

{{ Triumvirate }} uses the {py:class}`~triumvirate.parameters.ParameterSet` class
to handle parameter sets.

In [1]:
from triumvirate.parameters import ParameterSet

## Templates

As a starting point, {{ Triumvirate }} provides parameter templates which you
can modify. The templates come in two formats: as a a YAML file or 
a Python dictionary.

To fetch the templates, one could use
{py:func}`~triumvirate.parameters.fetch_paramset_template`, with the
argument ``'text'`` for the text-based YAML file and ``'dict'`` for
the Python dictionary.

In [2]:
from triumvirate.parameters import fetch_paramset_template 

### YAML file template

Let's first have a look at the content of the YAML file template,

In [3]:
parameter_template = fetch_paramset_template('text')

print(parameter_template)  # DEMO

---

# -- I/O -----------------------------------------------------------------

# Directories for input/output.
# The paths can be either absolute or relative to the working directory.
# If unset, the current working directory is assumed.
directories:
  catalogues:
  measurements:

# Filenames (with extensions) for input/output sources.
files:
  data_catalogue:
  rand_catalogue:

# Tags to be substituted into input/output paths.
tags:
  output: ~


# -- Mesh sampling -------------------------------------------------------

# Box size in each dimension (in Mpc/h). [mandatory]
boxsize:
  x:
  y:
  z:

# Grid cell number in each dimension. [mandatory]
ngrid:
  x:
  y:
  z:

# Box alignment: {'centre' (default), 'pad'}.
alignment: centre

# Padding scale: {'box' (default), 'grid'}.
padscale: box

# Padding factor (as a multiple of the size of padding scale).
padfactor: ~

# Mesh assignment scheme: {'ngp', 'cic', 'tsc' (default), 'pcs'}.
assignment: tsc

# Interlacing switch: {true/on, fal

which you can save to a `.yml` file (named ``"parameter_template.yml"``
in the current working directory below),

In [4]:
parameter_filepath = "parameter_template.yml"
with open(parameter_filepath, 'w') as parameter_file:
    parameter_file.write(parameter_template)

### Python dictionary template

Alternatively, one could use the Python dictionary template:

In [5]:
parameter_dict = fetch_paramset_template('dict')

# DEMO: format printing of dictionaries.
from pprint import pprint 
pprint(parameter_dict)

{'alignment': 'centre',
 'assignment': 'tsc',
 'binning': 'lin',
 'boxsize': {'x': None, 'y': None, 'z': None},
 'catalogue_type': None,
 'degrees': {'ELL': None, 'ell1': None, 'ell2': None},
 'directories': {'catalogues': None, 'measurements': None},
 'files': {'data_catalogue': None, 'rand_catalogue': None},
 'form': 'diag',
 'idx_bin': None,
 'interlace': False,
 'ngrid': {'x': None, 'y': None, 'z': None},
 'norm_convention': 'particle',
 'num_bins': None,
 'padfactor': None,
 'padscale': 'box',
 'range': [None, None],
 'statistic_type': None,
 'tags': {'output': None},
 'verbose': 20,
 'wa_orders': {'i': None, 'j': None}}


## Initialisation

### From a YAML file

To initialise a parameter set from the YAML file template, let's try:

In [6]:
from triumvirate.parameters import InvalidParameterError  # DEMO

try:
    paramset = ParameterSet(param_filepath=parameter_filepath)
except InvalidParameterError as exception:  # DEMO
    print(repr(exception))

InvalidParameterError('`boxsize` parameters must be set.')


As one could see, {py:class}`~triumvirate.parameters.ParameterSet` performs
validation checks on parameter values, and unset mandatory parameters
result in the exception {py:exc}`~triumvirate.parameters.InvalidParameterError`
above. To fix this, (externally) edit the file ``"parameter_template.yml"``
saved above to make sure all parameters marked by ``[mandatory]`` in the file
comments are set, e.g. with the following line changes:

```{code-block} yaml
:caption: parameter_template.yml

boxsize:
  x: 1000.
  y: 1000.
  z: 1000.
  
ngrid:
  x: 64
  y: 64
  z: 64
  
catalogue_type: sim

statistic_type: bispec

degrees:
  ell1: 0
  ell2: 0
  ELL: 0
  
range: [0.005, 0.105]

num_bins: 10
```

In [7]:
# DEMO: generally one would reproduce the steps below by editing the
# "parameter_template.yml" file externally.

def replace_yaml_template_parameter(str_original, str_new):
    global parameter_template
    parameter_template = parameter_template.replace(str_original, str_new)

parameter_replacements = [
    (
        "boxsize:\n  x:\n  y:\n  z:\n",
        "boxsize:\n  x: 1000.\n  y: 1000.\n  z: 1000.\n"
    ),
    (
        "ngrid:\n  x:\n  y:\n  z:\n",
        "ngrid:\n  x: 64\n  y: 64\n  z: 64\n"
    ),
    (
        "catalogue_type:",
        "catalogue_type: sim"
    ),
    (
        "statistic_type:",
        "statistic_type: bispec"
    ),
    (
        "ell1:",
        "ell1: 0"
    ),
    (
        "ell2:",
        "ell2: 0"
    ),
    (
        "ELL:",
        "ELL: 0"
    ),
    (
        "range: [~, ~]",
        "range: [0.005, 0.105]"
    ),
    (
        "num_bins:",
        "num_bins: 10"
    ),
]

for replacement in parameter_replacements:
    replace_yaml_template_parameter(*replacement)

with open(parameter_filepath, 'w') as parameter_file:
    parameter_file.write(parameter_template)

Now let's try to initialise the parameter set again:

In [8]:
parameter_set = ParameterSet(param_filepath=parameter_filepath)

[2023-03-05 23:35:27 (+00:00:01) STAT C++] Parameters validated.


### From a Python dictionary

Similarly, one can initialise the parameter set from the Python dictionary
template, but only after all the mandatory parameters have been set:

In [9]:
for ax_name in ['x', 'y', 'z']:
    parameter_dict['boxsize'][ax_name] = 1000.
    parameter_dict['ngrid'][ax_name] = 64.

# NOTE: 'ell1' and 'ell2' are only mandatory for
# three-point statistic algorithms.
parameter_dict.update({
    'catalogue_type': 'sim',
    'statistic_type': 'bispec',
    'degrees'       : {'ell1': 0, 'ell2': 0, 'ELL': 0},
    'range'         : [0.005, 0.105],
    'num_bins'      : 10,
})

In [10]:
parameter_set = ParameterSet(param_dict=parameter_dict)

[2023-03-05 23:35:27 (+00:00:01) STAT C++] Parameters validated.


## Parameter access

One can access individual parameters from
{py:class}`~triumvirate.parameters.ParameterSet` either as an attribute, or
like a dictionary by key or with the
{py:meth}`~triumvirate.parameters.ParameterSet.get` method:

In [11]:
print("Binning scheme:", parameter_set['binning'])
print("Binning scheme:", parameter_set.get('binning'))  # equivalent
print("Binning scheme:", parameter_set.binning)         # equivalent

Binning scheme: lin
Binning scheme: lin
Binning scheme: lin


One can also extract all parameters like a dictionary using the
{py:meth}`~triumvirate.parameters.ParameterSet.items` method:

In [12]:
pprint(dict(parameter_set.items()))

{'alignment': 'centre',
 'assignment': 'tsc',
 'binning': 'lin',
 'boxsize': {'x': 1000.0, 'y': 1000.0, 'z': 1000.0},
 'catalogue_type': 'sim',
 'degrees': {'ELL': 0, 'ell1': 0, 'ell2': 0},
 'directories': {'catalogues': None, 'measurements': None},
 'files': {'data_catalogue': None, 'rand_catalogue': None},
 'form': 'diag',
 'idx_bin': None,
 'interlace': 'false',
 'ngrid': {'x': 64.0, 'y': 64.0, 'z': 64.0},
 'norm_convention': 'particle',
 'npoint': '3pt',
 'num_bins': 10,
 'padfactor': None,
 'padscale': 'box',
 'range': [0.005, 0.105],
 'space': 'fourier',
 'statistic_type': 'bispec',
 'tags': {'output': None},
 'verbose': 20,
 'wa_orders': {'i': None, 'j': None}}


## Passing as an argument

Now whenever a callable in {{ Triumvirate }} accepts the `paramset` argument,
you can pass `parameter_set` above to it. Depending on the set-up, you may
need to make further modifications to your `parameter_set` to suit the data
and/or algorithm you are using.